# II. 영어 스팸 문자 데이터 전처리(PreProcess)

---
### 1) 데이터 로드 : dataframe (pandas module 사용)  
https://www.kaggle.com/uciml/sms-spam-collection-dataset  
train과 test가 별도로 나누어 지지 않았고, spam.csv 하나만 있다.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메시지 안보이게 설정

import gc
gc.collect() # garbage collector : 메모리 관리

In [ ]:
import os

---
전역 변수 중 일부(디렉토리 이름과 파일 이름 등)는 대문자로  
나머지 변수는 소문자로

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
DATA_DIR = 'data'
RAW_DATA_FILE = 'spam.csv'
RAW_DATA_PATH = os.path.join(DATA_DIR, RAW_DATA_FILE)

In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv(RAW_DATA_PATH, encoding='latin-1')

In [ ]:
# 결측 데이터는 없었으므로 바로 중복 데이터 제거
dedupe_raw_df = raw_df.drop_duplicates('v2', keep='first')

In [ ]:
dedupe_raw_df.head()

---
### 2) 라벨 인코딩 : ham을 0으로 spam을 1로 변환

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

dedupe_raw_df['label'] = le.fit_transform(dedupe_raw_df['v1'])

In [ ]:
dedupe_raw_df.head()

---
### 3) 데이터 분리 : training data와 test data로 분리

In [ ]:
# library for train test split
from sklearn.model_selection import train_test_split

In [ ]:
# Split data into train and test

test_size = 0.1 # split_ratio = ['0.9', '0.1']

# train_x, test_x, train_y, test_y = train_test_split(dedupe_raw_df['v2'], dedupe_raw_df['label'], test_size=0.1, random_state=434)
train_df, test_df = train_test_split(dedupe_raw_df, test_size=0.1, random_state=434)

---
#### (가) split한 데이터를 파일로 저장

In [ ]:
print(type(train_df))
print(type(test_df))

In [ ]:
train_df.head()

In [ ]:
# merged_train_df = pd.merge(train_x, train_y, left_index=True, right_index=True, how='left')
# merged_test_df = pd.merge(test_x, test_y, left_index=True, right_index=True, how='left')

In [ ]:
# DATA_DIR = 'data'
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

SPLITED_TRAIN_FILE = 'splited_train.csv'
SPLITED_TEST_FILE = 'splited_test.csv'

SPLITED_TRAIN_PATH = os.path.join(PROCESSED_DATA_DIR, SPLITED_TRAIN_FILE)
SPLITED_TEST_PATH = os.path.join(PROCESSED_DATA_DIR, SPLITED_TEST_FILE)

In [ ]:
train_df.to_csv(SPLITED_TRAIN_PATH, index = False)
test_df.to_csv(SPLITED_TEST_PATH, index = False)

---
### 4) 토큰화(Tokenization) : 문장을 단어 조각으로 분리하는 것

In [ ]:
train_x = train_df['v2']
train_y = train_df['label']
test_x = test_df['v2']
test_y = test_df['label']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing import sequence

---
(가) 단어 사전을 만들기 위해 첫 번째 토큰화 작업
  - 디폴트 값으로 토큰화하여 결과를 관찰하고 참고 한다.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x)  # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합(vocabulary)을 생성한다.

In [ ]:
# 분석한 결과를 보기 : 전체 어휘의 숫자와 희귀단어 분석

threshold = 3 # 등장 빈도 하한, 이 수보다 작은 빈도를 나타내는 어휘는 희귀 단어로 정한다.
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 어휘 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

---
(나) 앞에서 만든 단어 사전을 이용하여 새롭게 토큰화
  - Tokenizer()를 초기화하고 다시 토큰화 한다.

In [ ]:
# vocab_size = 2534

oov_tok = "<OOV>" # Out-Of-Vocabulary 토큰

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_x)

In [ ]:
word_index = tokenizer.word_index
word_index

In [ ]:
# 밸류로 정렬, 사용 빈도가 많은 어휘부터 볼 수 있다.
import operator

sorted_dict = sorted(tokenizer.word_counts.items(), key=operator.itemgetter(1), reverse=True)
sorted_dict

In [ ]:
oov_tok = "<OOV>" 

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_x)

In [ ]:
# check how many words 
tot_words = len(tokenizer.word_index)
print('There are %s unique tokens in training data. ' % tot_words)

In [ ]:
TOKENIZED_FILE = 'tokenized.json'
TOKENIZED_PATH = os.path.join(DATA_DIR, TOKENIZED_FILE)

import json

tok_json = tokenizer.to_json()
# with io.open(DATA_IN_DIR + 'tokenizer_' + ver + '.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(tokenizer_json, ensure_ascii=False))

json.dump(tok_json, open(TOKENIZED_PATH, 'w'), ensure_ascii=False)

# 읽는 방법 : 아래 2 가지 방법 중 한 가지 방법으로 읽어 올 수 있다.
'''
tok_configs = None
with open(tokenized_file, 'r') as f:
    tok_configs = json.load(f)
'''
# keras.preprocessing.text.tokenizer_from_json(json_string)
'''
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
'''

---
### 5) 시퀀싱과 패딩(Sequencing and Padding)  
토큰화 된 데이터를 숫자(인덱스) 시퀀스로 바꾸고 같은 길이의 시퀀스로 만들기 위해 패딩을 한다.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_x # split할 때 순서가 바뀌었음.

In [ ]:
train_y

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_x)

In [ ]:
train_sequences

In [ ]:
# Defining pre-processing hyperparameters
max_len = 20 
trunc_type = "post" 
padding_type = "post" 

# padding : max_len에 길이를 맞춘다. 길면 자르고 부족하면 0으로 채운다.
train_sequences = tokenizer.texts_to_sequences(train_x)
train_padded = pad_sequences (train_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )

In [ ]:
train_x[2901] # 첫 번째 데이터의 인덱스가 2901

In [ ]:
train_sequences[0] # 확인

In [ ]:
text = tokenizer.sequences_to_texts([train_sequences[0]]) # 역으로 문장 확인
text

In [ ]:
# Shape of train tensor
print('Shape of train tensor: ', train_padded.shape)

In [ ]:
# Before padding
len(train_sequences[0]), len(train_sequences[1])

In [ ]:
# After padding
len(train_padded[0]), len(train_padded[1])

In [ ]:
print(train_padded[0])

---
### 6) 전처리(PreProcessed)된 Data를 파일에 저장

dataframe의 객체를 그대로 저장하려면, (list, dict 등 포함) pickle로 저장한다.    
to_pickle, read_pickle도 사용 가능 :   
https://wikidocs.net/8929  
https://tariat.tistory.com/739  

In [ ]:
# DATA_DIR = 'data'
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

PROCESSED_DATA_FILE = "train_padded.p"
PROCESSED_LABEL_FILE = "train_label.p"

PROCESSED_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_DATA_FILE)
PROCESSED_LABEL_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_LABEL_FILE)

In [ ]:
train_y

In [ ]:
# 데이터 저장, pickle을 이용하면 python의 데이터 타입을 그대로 저장할 수 있음.
import pickle

with open(PROCESSED_DATA_PATH, "wb") as file:
    pickle.dump(train_padded, file)
    
with open(PROCESSED_LABEL_PATH, "wb") as file:
    pickle.dump(train_y, file)

In [ ]:
# 저장된 파일 로드 확인
with open(PROCESSED_DATA_PATH, "rb" ) as file:
    train_padded_loaded = pickle.load(file)

In [ ]:
train_padded_loaded

In [ ]:
train_padded_loaded.shape

---
### 7) Test dataset 전처리

트레이닝 후 정확도를 측정하기 위해 사용하기 위해 테스트 데이터셋도 전처리해 둔다.
  1. test dataset load : split 해서 얻은 데이터셋 사용
  2. 토큰화
  3. 시퀀스 만들기와 패딩

In [ ]:
test_x

In [ ]:
test_y

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_x)
test_padded = pad_sequences(test_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)

In [ ]:
print('Shape of testing tensor: ', test_padded.shape)

In [ ]:
# DATA_DIR = 'data'
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

PROCESSED_TEST_DATA_FILE = "test_padded.p"
PROCESSED_TEST_LABEL_FILE = "test_label.p"

PROCESSED_TEST_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_TEST_DATA_FILE)
PROCESSED_TEST_LABEL_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_TEST_LABEL_FILE)

In [ ]:
# 데이터 저장, pickle을 이용하면 python의 데이터 타입을 그대로 저장할 수 있음.
import pickle

with open(PROCESSED_TEST_DATA_PATH, "wb") as file:
    pickle.dump(test_padded, file)
    
with open(PROCESSED_TEST_LABEL_PATH, "wb") as file:
    pickle.dump(test_y, file)